# M1 useful nodes (Broken)
- Developed on Google Colab using an A100 with 40GB GPU and 80GB system RAM.
- Runs with GPT2/TinyStories/Qwen/Llama/Granite/SmolLM with base/CS1/CS2/CS3.
- Requires a GITHUB_TOKEN secret to access Martian quanta_text_to_sql code repository.
- Requires a HF_TOKEN secret to access Martian HuggingFace repository.


# Import libraries
Imports standard libraries. Do not read.

In [ ]:
# https://nnsight.net/
!pip install -U nnsight

In [ ]:
from IPython.display import clear_output
import einops
import torch
import numpy as np
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "colab"

import nnsight
from nnsight import LanguageModel, util
from nnsight.tracing.Proxy import Proxy

In [ ]:
from getpass import getpass
from google.colab import userdata
import gc
import weakref

In [ ]:
github_token = userdata.get("GITHUB_TOKEN")

# Install the private repository using the token
!pip install --upgrade git+https://{github_token}@github.com/withmartian/quanta_text_to_sql.git

import QuantaTextToSql as qts

# Select model, command set and feature to investigate


In [ ]:
model_num = 1                 # 0=GPT2, 1=TinyStories, 2=Qwen, 3=Llama, 4=Granite, 5=SmolLM
cs_num = 1                    # 0=BaseModel, 1=CS1, 2=CS2 or 3=CS3
max_new_tokens = 100          # Max number of tokens to generate

# Run m1, m2 and m3 models

In [ ]:
if model_num > 0:

    if model_num == 1:
        the_tokenizer, the_model = qts.load_sql_interp_model(model_num, cs_num, auth_token=userdata.get("HF_TOKEN"), use_flash_attention=False)
        model = LanguageModel(the_model, the_tokenizer)
        model.tokenizer = the_tokenizer
    else:
        model = LanguageModel(qts.sql_interp_model_location(model_num, cs_num), device_map="auto")


    clear_output()
    print(model)

In [ ]:
if model_num > 0:
    # Generate a batch of prompts
    batch_size = 3

    if cs_num == 0 or cs_num == 1:
      examples = qts.generate_cs1(batch_size)
    elif cs_num == 2:
      examples = qts.generate_cs2(batch_size)
    elif cs_num == 3:
      examples = qts.generate_cs3(batch_size)

In [ ]:
def calc_max_prompt_tokens(examples):

    max_prompt_tokens = 0
    for example in examples:
        prompt = example.get_alpaca_prompt()
        tokens = model.tokenizer(prompt)["input_ids"]
        max_prompt_tokens = max(max_prompt_tokens, len(tokens))

    return max_prompt_tokens

In [ ]:
N_LAYERS = len(model.transformer.h)
max_prompt_tokens = calc_max_prompt_tokens(examples)
print("Sizes:", batch_size, N_LAYERS, max_prompt_tokens)

In [ ]:
# Identify the nodes that, when ablating, cause a decrease in model prediction accuracy.
def zero_activation_experiment(examples, model):

    # https://nnsight.net/notebooks/tutorials/walkthrough/#Batching
    # https://nnsight.net/notebooks/tutorials/walkthrough/#Looping
    with model.session(remote=True) as session:

        # Generate the list of experiments to run
        run_list = nnsight.list()
        for example in examples:
            prompt = example.get_alpaca_prompt()
            answer = example.sql_statement
            num_prompt_tokens = len(model.tokenizer(prompt)["input_ids"])
            for layer_idx in range(N_LAYERS):
                for token_idx in range(num_prompt_tokens):
                    run_list.append([prompt, answer, layer_idx, token_idx])

        # Iterate through the experiments
        with session.iter(run_list, return_context=True) as (item, iterator):
            iterator.log(item)

            prompt = item.node[0] # Fails
            answer = item.node[1]
            run_layer_idx = item.node[2]
            run_token_idx = item.node[3]

            with model.trace(prompt):
                # Zero the activations at the given layer and token position
                model.transformer.h[run_layer_idx].output[0][:, run_token_idx, :] = 0

                patched_logits = model.lm_head.output.save()

                # Get (impacted) model output
                final_output = model.lm_head.output
                final_output = final_output.argmax(dim=-1)
                decoded_output = model.tokenizer.decode(final_output[0], skip_special_tokens=True)

                # Check if the output changed
                if prompt + answer != decoded_output:
                    results_summary[layer_idx, token_idx] += 1

    # Calculate percentage of output changes for each layer and token position
    results_percentage = (results_summary / len(examples)) * 100

    return results_percentage

# Example Usage
results = zero_activation_experiment(examples, model)

In [ ]:
# Print summary of results
import matplotlib.pyplot as plt
plt.imshow(results, cmap="viridis", aspect="auto")
plt.colorbar(label="Percentage Change")
plt.xlabel("Token Position")
plt.ylabel("Layer")
plt.title("Percentage of Output Changes by Zeroing Activations")
plt.show()